# SAP ERPよりBAPI_COMPANYCODE_GETLISTでデータ取得

In [1]:
from pyrfc import Connection
sap_conn = Connection(
    ashost='xxx.xxx.xxx.xxx',  # IP or HOSTNAME
    sysnr='00', 
    client='800', 
    user='idadmin', 
    passwd='xxxx',
)
res = sap_conn.call('BAPI_COMPANYCODE_GETLIST')
company_code_list = res['COMPANYCODE_LIST']

# 最初の10件を表示
company_code_list[:10]

[{'COMP_CODE': '0001', 'COMP_NAME': 'SAP A.G.'},
 {'COMP_CODE': '0005', 'COMP_NAME': 'BestRun Germany new GL'},
 {'COMP_CODE': '0006', 'COMP_NAME': 'BestRun US INC New GL'},
 {'COMP_CODE': '0007', 'COMP_NAME': 'BestRun NEW GL'},
 {'COMP_CODE': '0008', 'COMP_NAME': 'BestRun US INC New GL 8'},
 {'COMP_CODE': '1000', 'COMP_NAME': 'Best Run (do not change!)'},
 {'COMP_CODE': '1002', 'COMP_NAME': 'BestRun Singapore'},
 {'COMP_CODE': '1005', 'COMP_NAME': 'Subsidiary Germany'},
 {'COMP_CODE': '1109', 'COMP_NAME': 'AB Tetra Pak'},
 {'COMP_CODE': '2000', 'COMP_NAME': 'BestRun UK'}]

# PandasにSAP ERPのデータを読み込み

In [2]:
import pandas as pd
df = pd.DataFrame(
    data=company_code_list,
    columns={"COMP_CODE","COMP_NAME"},
    )
df = df.rename(columns={"COMP_CODE":"company_code","COMP_NAME":"company_name"})

# Dataframeに読み込んだデータの最初の10件表示
df.head(10)

,company_code,company_name
0,0001,SAP A.G.
1,0005,BestRun Germany new GL
2,0006,BestRun US INC New GL
3,0007,BestRun NEW GL
4,0008,BestRun US INC New GL 8
5,1000,Best Run (do not change!)
6,1002,BestRun Singapore
7,1005,Subsidiary Germany
8,1109,AB Tetra Pak
9,2000,BestRun UK


# BigQueryにロード

In [3]:
import pydata_google_auth
cred = pydata_google_auth.get_user_credentials(
    ['https://www.googleapis.com/auth/bigquery'],
)

In [9]:
from google.cloud import bigquery

# パラメータを設定
PROJECT_ID = 'your-project'
TABLE_ID = 'your_dataset.your_company_code_table'

# DataframeからBigQueryにロード
client = bigquery.Client(project=PROJECT_ID,credentials=cred)
job_config = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField("company_code", "STRING"),
        bigquery.SchemaField("company_name", "STRING"),
    ],
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE)

# DataframeをBigQueryに書き出し
job = client.load_table_from_dataframe(
    df, TABLE_ID, job_config=job_config
)

# ロードジョブの完了を待つ
import time
while not job.done():
    print(job.state)
    time.sleep(1)
print(job.state)
if job.errors:
    print(f"Error ->{job.errors}")

RUNNING
RUNNING
RUNNING
RUNNING
DONE
